# ETAPA 2a — Preparação do Painel CAGED + ILO Exposure Index

**Dissertação:** Inteligência Artificial Generativa e o Mercado de Trabalho Brasileiro: Uma Análise de Exposição Ocupacional e seus Efeitos Distributivos.

**Aluno:** Manoel Brasil Orlandi

---

### Contextualização

A rápida difusão de modelos de IA generativa (LLMs, geradores de imagem/código) levanta questões centrais sobre seus impactos no mercado de trabalho. Para mensurar esse potencial de impacto, a Organização Internacional do Trabalho (OIT) criou um índice de exposição ocupacional à IA generativa, publicado como *Working Paper* 140 (WP140). O índice atribui scores de exposição a cada ocupação da classificação ISCO-08, com base na avaliação de suas tarefas constituintes por modelos de linguagem e validação humana.

Este notebook prepara uma base de dados que junta os dados do **Novo CAGED** (Cadastro Geral de Empregados e Desempregados) ao **índice de exposição à IA generativa da OIT**, para ser usado em um modelo de Diferenças-em-Diferenças (DiD) no Notebook 2b.

### Objetivo

Construir o **painel mensal de ocupações formais brasileiras (2021–2025)** a partir do Novo CAGED, realizar o **crosswalk CBO 2002 → ISCO-08** (especificação dual: 2 dígitos como principal, 4 dígitos para robustez), e fazer o merge com o índice de exposição à IA generativa da OIT (Gmyrek, Berg & Cappelli, 2025). O output final é um dataset analítico pronto para a estimação DiD.

**Estratégia de crosswalk:** Análise principal a **2 dígitos** ISCO-08 (match por Sub-major Group com fallback hierárquico a Major Group), com robustez a **4 dígitos** via correspondência ISCO-88 ↔ ISCO-08 + fallback hierárquico em 6 níveis.

**Inspiração metodológica:** Hui, Reshef & Zhou (2024), "The Short-Term Effects of Generative Artificial Intelligence on Employment: Evidence from an Online Labor Market" — adaptado para dados administrativos brasileiros (CAGED) com o índice ILO de exposição ocupacional.

### Ficha Técnica dos Dados

| Item | Descrição |
|------|-----------|
| **Fonte CAGED** | Ministério do Trabalho e Emprego (MTE), via Base dos Dados (BigQuery) |
| **Dataset BigQuery** | `basedosdados.br_me_caged.microdados_movimentacao` |
| **Período** | Janeiro/2021 — Dezembro/2025 (60 meses) |
| **Unidade** | Movimentação individual (admissão ou desligamento) |
| **Cobertura** | Emprego formal (CLT) em todo o Brasil |
| **Índice ILO** | `ilo_exposure_clean.csv` — 427 ocupações ISCO-08 com exposure scores |
| **Classificação** | CBO 2002 (CAGED) → ISCO-08 (ILO) via crosswalk hierárquico |

### Referências principais
- Gmyrek, P., Berg, J. & Cappelli, D. (2025). *Generative AI and Jobs: An updated global assessment*. ILO Working Paper 140.
- Hui, X., Reshef, O. & Zhou, L. (2024). *The Short-Term Effects of Generative AI on Employment*. Organization Science, 35(6).
- Brynjolfsson, E., Chandar, P. & Chen, J. (2025). *Canaries in the Coal Mine? Six Facts about the Recent Employment Effects of AI*.
- Callaway, B. & Sant'Anna, P. (2021). *Difference-in-differences with multiple time periods*. Journal of Econometrics, 225(2).
- Muendler, M.-A. & Poole, J.P. (2004). *Job Concordances for Brazil: Mapping CBO to ISCO-88*. UC San Diego.


### 1. Configuração do ambiente

Definir caminhos, importar bibliotecas e configurar parâmetros do painel. Todos os caminhos são relativos ao diretório `notebook/`.

> **Nota sobre a janela temporal:** Excluímos 2020 para evitar os efeitos distorcivos da pandemia de COVID-19 sobre o mercado de trabalho formal. O ano de 2020 apresentou quedas e recuperações atípicas que contaminariam o período pré-tratamento do DiD. A janela Jan/2021–Dez/2025 oferece 23 meses pré-ChatGPT e 31 meses pós.

> **Nota sobre o Novo CAGED:** A partir de janeiro/2020, o CAGED foi substituído pelo sistema eSocial (Portaria SEPRT 1.127/2019). Usamos dados de 2021+ para consistência metodológica (eSocial já estabilizado).

In [1]:
# Etapa 2a.1 — Configuração do ambiente

import warnings
import pandas as pd
import numpy as np
from pathlib import Path

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------------------------------------
# Caminhos (relativos ao diretório do notebook)
# ---------------------------------------------------------------------------
DATA_INPUT     = Path("data/input")
DATA_RAW       = Path("data/raw")
DATA_PROCESSED = Path("data/processed")
DATA_OUTPUT    = Path("data/output")

for d in [DATA_INPUT, DATA_RAW, DATA_PROCESSED, DATA_OUTPUT]:
    d.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------------------
# Parâmetros do Painel CAGED
# ---------------------------------------------------------------------------
GCP_PROJECT_ID = "mestrado-pnad-2026"

ANO_INICIO     = 2021
ANO_FIM        = 2025
ANO_TRATAMENTO = 2022
MES_TRATAMENTO = 12   # Dezembro/2022 como primeiro mês "pós"

SALARIO_MINIMO = {
    2021: 1100, 2022: 1212, 2023: 1320, 2024: 1412, 2025: 1518
}

# ---------------------------------------------------------------------------
# Colunas a selecionar do CAGED (BigQuery)
# ---------------------------------------------------------------------------
COLUNAS_CAGED = """
    ano, mes, sigla_uf, id_municipio, cbo_2002,
    categoria, tipo_movimentacao, saldo_movimentacao,
    salario_mensal, grau_instrucao, idade, sexo, raca_cor,
    cnae_2_secao, cnae_2_subclasse, tamanho_estabelecimento_janeiro
"""

# ---------------------------------------------------------------------------
# Arquivos de referência
# ---------------------------------------------------------------------------
ILO_FILE               = DATA_PROCESSED / "ilo_exposure_clean.csv"
ISCO_08_88_FILE        = DATA_INPUT / "Correspondência ISCO 08 a 88.xlsx"
ISCO_08_ESTRUTURA_FILE = DATA_INPUT / "ISCO 08 Estruturas e Definições.xlsx"
MUENDLER_FILE          = DATA_INPUT / "cbo-isco-conc.csv"

# ---------------------------------------------------------------------------
# Checkpoints intermediários
# ---------------------------------------------------------------------------
PAINEL_MENSAL_FILE     = DATA_PROCESSED / "painel_caged_mensal.parquet"
PAINEL_CROSSWALK_FILE  = DATA_PROCESSED / "painel_caged_crosswalk.parquet"
PAINEL_TRATAMENTO_FILE = DATA_PROCESSED / "painel_caged_tratamento.parquet"
PAINEL_FINAL_PARQUET   = DATA_OUTPUT / "painel_caged_did_ready.parquet"
PAINEL_FINAL_CSV       = DATA_OUTPUT / "painel_caged_did_ready.csv"

# ---------------------------------------------------------------------------
# Grandes grupos CBO (para sanity checks)
# ---------------------------------------------------------------------------
GRANDES_GRUPOS_CBO = {
    '0': 'Forças Armadas',
    '1': 'Dirigentes',
    '2': 'Profissionais das ciências',
    '3': 'Técnicos nível médio',
    '4': 'Trabalhadores de serv. admin.',
    '5': 'Trabalhadores de serviços/comércio',
    '6': 'Agropecuária',
    '7': 'Produção industrial',
    '8': 'Operadores de máquinas',
    '9': 'Manutenção e reparação',
}

print("Configuração carregada.")
print(f"  Período: {ANO_INICIO}–{ANO_FIM} ({(ANO_FIM - ANO_INICIO + 1) * 12} meses)")
print(f"  Evento: ChatGPT — Nov/2022 (pós a partir de {MES_TRATAMENTO}/{ANO_TRATAMENTO})")
print(f"  Projeto GCP: {GCP_PROJECT_ID}")
print(f"  ILO file: {ILO_FILE} (existe: {ILO_FILE.exists()})")

Configuração carregada.
  Período: 2021–2025 (60 meses)
  Evento: ChatGPT — Nov/2022 (pós a partir de 12/2022)
  Projeto GCP: mestrado-pnad-2026
  ILO file: data/processed/ilo_exposure_clean.csv (existe: True)


### 2a. Download dos microdados CAGED

Extrair do Novo CAGED (BigQuery/Base dos Dados) todas as movimentações de emprego formal no período 2021–2025.

| Item | Descrição |
|------|-----------|
| **Tabela BigQuery** | `basedosdados.br_me_caged.microdados_movimentacao` |
| **Período** | 2021-01 a 2025-12 |
| **Filtros** | `ano BETWEEN 2021 AND 2025` |
| **Volume estimado** | ~20-30M de registros por ano, ~100-150M total |
| **Estratégia** | Download ano a ano via `google-cloud-bigquery` (Storage API) com fallback para `basedosdados` |

> **Nota metodológica — Volume de dados:** O CAGED registra ~20-25 milhões de movimentações/ano. Para 5 anos, esperamos ~100-125M de registros. O download é feito ano a ano para evitar OOM e timeout, com salvamento em parquets individuais (`caged_{ano}.parquet`).

> **Nota sobre otimização:** Usamos a BigQuery Storage API (`create_bqstorage_client=True`) que transfere dados via gRPC/Arrow, sendo 2-5x mais rápida que o método padrão REST. Se o arquivo parquet já existir, o download é pulado automaticamente.

In [2]:
# Etapa 2a.2a — Download dos microdados CAGED
# Estratégia: download ano a ano via BigQuery Storage API, com cache local em parquet.

from google.cloud import bigquery

def download_caged_ano(ano):
    """Baixar microdados CAGED de um ano via BigQuery Storage API."""
    parquet_path = DATA_RAW / f"caged_{ano}.parquet"

    if parquet_path.exists():
        size_mb = parquet_path.stat().st_size / 1e6
        df = pd.read_parquet(parquet_path)
        print(f"  {ano}: Carregado do cache — {len(df):,} registros ({size_mb:.0f} MB)")
        return df

    print(f"  {ano}: Baixando do BigQuery...", end="", flush=True)
    query = f"""
    SELECT {COLUNAS_CAGED}
    FROM `basedosdados.br_me_caged.microdados_movimentacao`
    WHERE ano = {ano}
    """
    client = bigquery.Client(project=GCP_PROJECT_ID)
    df = client.query(query).to_dataframe(create_bqstorage_client=True)
    df.to_parquet(parquet_path, index=False)
    size_mb = parquet_path.stat().st_size / 1e6
    print(f" {len(df):,} registros ({size_mb:.0f} MB)")
    return df

# ---------------------------------------------------------------------------
# Download ano a ano
# ---------------------------------------------------------------------------
print("Download dos microdados CAGED:")
dfs_anuais = []
for ano in range(ANO_INICIO, ANO_FIM + 1):
    df_ano = download_caged_ano(ano)
    dfs_anuais.append(df_ano)

# Resumo (sem concatenar em memória para evitar OOM)
total = sum(len(df) for df in dfs_anuais)
print(f"\nTotal: {total:,} movimentações ({ANO_INICIO}–{ANO_FIM})")
print(f"Colunas: {list(dfs_anuais[0].columns)}")

Download dos microdados CAGED:
  2021: Carregado do cache — 36,554,795 registros (384 MB)
  2022: Carregado do cache — 42,475,516 registros (445 MB)
  2023: Carregado do cache — 44,485,982 registros (466 MB)
  2024: Carregado do cache — 48,996,040 registros (511 MB)
  2025: Carregado do cache — 26,312,103 registros (269 MB)

Total: 198,824,436 movimentações (2021–2025)
Colunas: ['ano', 'mes', 'sigla_uf', 'id_municipio', 'cbo_2002', 'categoria', 'tipo_movimentacao', 'saldo_movimentacao', 'salario_mensal', 'grau_instrucao', 'idade', 'sexo', 'raca_cor', 'cnae_2_secao', 'cnae_2_subclasse', 'tamanho_estabelecimento_janeiro']


### 2b. Verificar dados CAGED (CHECKPOINT)

Verificar integridade dos dados baixados: cobertura temporal (12 meses/ano), volume por ano, preenchimento de variáveis-chave, e formato dos códigos CBO.

**Critérios de aceite:**
- Todos os meses cobertos (Jan–Dez) para cada ano
- CBO com >95% de preenchimento
- ~20-30M registros por ano


In [3]:
# Etapa 2a.2b — CHECKPOINT: Verificar dados CAGED
# Carrega cada parquet individualmente (para evitar OOM)

print("=" * 60)
print("CHECKPOINT — Microdados CAGED")
print("=" * 60)

total_registros = 0
for ano in range(ANO_INICIO, ANO_FIM + 1):
    parquet_path = DATA_RAW / f"caged_{ano}.parquet"
    df = pd.read_parquet(parquet_path)

    # Volume
    print(f"\n--- {ano}: {len(df):,} movimentações ---")
    total_registros += len(df)

    # Cobertura mensal
    meses = sorted(df['mes'].dropna().unique())
    status = "OK" if len(meses) == 12 else f"ALERTA: {len(meses)} meses"
    print(f"  Meses: {len(meses)} ({status})")

    # Preenchimento CBO
    cbo_pct = df['cbo_2002'].notna().mean()
    print(f"  CBO preenchido: {cbo_pct:.1%}")

    # CBOs únicos
    cbos = df['cbo_2002'].dropna().astype(str).str[:4].nunique()
    print(f"  Famílias CBO 4d únicas: {cbos}")

    # Admissões vs desligamentos
    if 'saldo_movimentacao' in df.columns:
        adm = (df['saldo_movimentacao'] == 1).sum()
        desl = (df['saldo_movimentacao'] == -1).sum()
        print(f"  Admissões: {adm:,} | Desligamentos: {desl:,} | Saldo: {adm-desl:+,}")

print(f"\n{'=' * 60}")
print(f"TOTAL: {total_registros:,} movimentações ({ANO_INICIO}–{ANO_FIM})")
print(f"{'=' * 60}")

CHECKPOINT — Microdados CAGED

--- 2021: 36,554,795 movimentações ---
  Meses: 12 (OK)
  CBO preenchido: 100.0%
  Famílias CBO 4d únicas: 626
  Admissões: 19,703,604 | Desligamentos: 16,851,191 | Saldo: +2,852,413

--- 2022: 42,475,516 movimentações ---
  Meses: 12 (OK)
  CBO preenchido: 100.0%
  Famílias CBO 4d únicas: 624
  Admissões: 22,243,441 | Desligamentos: 20,232,075 | Saldo: +2,011,366

--- 2023: 44,485,982 movimentações ---
  Meses: 12 (OK)
  CBO preenchido: 100.0%
  Famílias CBO 4d únicas: 626
  Admissões: 22,982,161 | Desligamentos: 21,503,821 | Saldo: +1,478,340

--- 2024: 48,996,040 movimentações ---
  Meses: 12 (OK)
  CBO preenchido: 100.0%
  Famílias CBO 4d únicas: 625
  Admissões: 25,336,277 | Desligamentos: 23,659,763 | Saldo: +1,676,514

--- 2025: 26,312,103 movimentações ---
  Meses: 6 (ALERTA: 6 meses)
  CBO preenchido: 100.0%
  Famílias CBO 4d únicas: 622
  Admissões: 13,763,059 | Desligamentos: 12,549,044 | Saldo: +1,214,015

TOTAL: 198,824,436 movimentações (202

### 3a. Agregação: Microdados → Painel Mensal por Ocupação

Agregar os microdados de movimentação (nível individual) em um painel mensal por ocupação CBO (4 dígitos). Cada linha do painel representará uma ocupação-mês com métricas agregadas.

#### Estratégia de agregação

Seguindo a abordagem de Hui, Reshef & Zhou (2024), construímos um painel ao nível de **ocupação × mês** com as seguintes métricas:

| Métrica | Cálculo | Descrição |
|---------|---------|-----------|
| `admissoes` | Contagem de `saldo_movimentacao == 1` | Fluxo de contratação |
| `desligamentos` | Contagem de `saldo_movimentacao == -1` | Fluxo de demissão |
| `saldo` | `admissoes - desligamentos` | Criação líquida de empregos |
| `salario_medio_adm` | Média do `salario_mensal` (admissões) | Nível salarial |
| `salario_mediano_adm` | Mediana do `salario_mensal` (admissões) | Robustez a outliers |
| `pct_mulher_adm` | % de `sexo == 3` nas admissões | Composição de gênero |
| `pct_superior_adm` | % com `grau_instrucao >= 9` | Composição educacional |

> **Nota — CBO 4 dígitos:** A CBO tem 6 dígitos (XXXX-XX), onde os 4 primeiros definem a "família" ocupacional. Para o crosswalk com ISCO-08, usamos os 4 primeiros dígitos (família CBO ≈ unit group ISCO-08).

> **Nota — Otimização de memória:** O processamento é feito ano a ano para evitar OOM (Out-of-Memory) com ~100M+ registros. Flags booleanos são pré-computados como float para permitir agregação vetorizada (evitando lambdas lentas). A mediana é calculada separadamente por ser computacionalmente cara.


In [4]:
# Etapa 2a.3a — Agregação: Microdados → Painel Mensal por Ocupação
# Checkpoint: se o painel já existe, carrega direto.

if PAINEL_MENSAL_FILE.exists():
    painel = pd.read_parquet(PAINEL_MENSAL_FILE)
    print(f"Painel carregado do checkpoint: {PAINEL_MENSAL_FILE.name}")
    print(f"  {len(painel):,} linhas, {painel['cbo_4d'].nunique()} ocupações, "
          f"{painel['periodo'].nunique()} períodos")
else:
    print("Construindo painel a partir dos microdados (ano a ano)...")
    paineis_anuais = []

    for ano in range(ANO_INICIO, ANO_FIM + 1):
        print(f"\n  Processando {ano}...", flush=True)
        df = pd.read_parquet(DATA_RAW / f"caged_{ano}.parquet")

        # CBO 4 dígitos
        df['cbo_2002'] = df['cbo_2002'].astype(str).str.strip()
        df['cbo_4d'] = df['cbo_2002'].str[:4]
        df = df[df['cbo_4d'].str.len() == 4]
        df = df[df['cbo_4d'].str.isdigit()]
        df = df[~df['cbo_4d'].isin(['0000'])]

        # Variáveis temporais
        df['periodo'] = df['ano'].astype(str) + '-' + df['mes'].astype(str).str.zfill(2)
        df['periodo_num'] = df['ano'].astype(int) * 100 + df['mes'].astype(int)
        df['post'] = (df['periodo_num'] >= ANO_TRATAMENTO * 100 + MES_TRATAMENTO).astype(int)

        # Flags booleanos pré-computados (para agregação vetorizada)
        df['is_mulher'] = (df['sexo'] == 3).astype(float)
        df['is_superior'] = df['grau_instrucao'].astype(str).isin(
            ['9', '10', '11', '12', '13']
        ).astype(float)

        # Separar admissões e desligamentos
        df_adm = df[df['saldo_movimentacao'] == 1]
        df_desl = df[df['saldo_movimentacao'] == -1]
        print(f"    {ano}: {len(df_adm):,} admissões, {len(df_desl):,} desligamentos", flush=True)

        # Agregar admissões (sem lambdas — vetorizado)
        painel_adm = df_adm.groupby(['cbo_4d', 'ano', 'mes']).agg(
            admissoes=('saldo_movimentacao', 'count'),
            salario_medio_adm=('salario_mensal', 'mean'),
            idade_media_adm=('idade', 'mean'),
            pct_mulher_adm=('is_mulher', 'mean'),
            pct_superior_adm=('is_superior', 'mean'),
        ).reset_index()

        # Mediana separada (performance)
        mediana = df_adm.groupby(['cbo_4d', 'ano', 'mes'])['salario_mensal'].median().reset_index()
        mediana.columns = ['cbo_4d', 'ano', 'mes', 'salario_mediano_adm']
        painel_adm = painel_adm.merge(mediana, on=['cbo_4d', 'ano', 'mes'], how='left')

        # Agregar desligamentos
        painel_desl = df_desl.groupby(['cbo_4d', 'ano', 'mes']).agg(
            desligamentos=('saldo_movimentacao', 'count'),
            salario_medio_desl=('salario_mensal', 'mean'),
        ).reset_index()

        # Merge
        p = painel_adm.merge(painel_desl, on=['cbo_4d', 'ano', 'mes'], how='outer').fillna(0)
        p['saldo'] = p['admissoes'] - p['desligamentos']
        p['n_movimentacoes'] = p['admissoes'] + p['desligamentos']
        p['periodo'] = p['ano'].astype(int).astype(str) + '-' + p['mes'].astype(int).astype(str).str.zfill(2)
        p['periodo_num'] = p['ano'].astype(int) * 100 + p['mes'].astype(int)
        p['post'] = (p['periodo_num'] >= ANO_TRATAMENTO * 100 + MES_TRATAMENTO).astype(int)
        p['ln_admissoes'] = np.log(p['admissoes'] + 1)
        p['ln_desligamentos'] = np.log(p['desligamentos'] + 1)
        p['ln_salario_adm'] = np.log(p['salario_medio_adm'].clip(lower=1))
        p['cbo_2d'] = p['cbo_4d'].str[:2]

        paineis_anuais.append(p)
        print(f"    → {len(p):,} linhas no painel", flush=True)

    painel = pd.concat(paineis_anuais, ignore_index=True)
    painel.to_parquet(PAINEL_MENSAL_FILE, index=False)
    print(f"\nPainel salvo: {PAINEL_MENSAL_FILE.name}")

print(f"\nPainel final: {len(painel):,} linhas")
print(f"  Ocupações: {painel['cbo_4d'].nunique()}, Períodos: {painel['periodo'].nunique()}")
print(f"  Shape: {painel.shape}")

Painel carregado do checkpoint: painel_caged_mensal.parquet
  32,988 linhas, 629 ocupações, 54 períodos

Painel final: 32,988 linhas
  Ocupações: 629, Períodos: 54
  Shape: (32988, 20)


### 3b. Verificar painel agregado (CHECKPOINT)

Verificar integridade do painel: dimensões, balanceamento (ocupações × períodos), cobertura temporal, distribuição de variáveis-chave e série temporal agregada.


In [5]:
# Etapa 2a.3b — CHECKPOINT: Verificar painel agregado

print("=" * 60)
print("CHECKPOINT — Painel Ocupação × Mês")
print("=" * 60)

# 1. Dimensões
n_ocup = painel['cbo_4d'].nunique()
n_periodos = painel['periodo'].nunique()
print(f"\nOcupações: {n_ocup}")
print(f"Períodos: {n_periodos}")
print(f"Painel teórico (balanceado): {n_ocup * n_periodos:,}")
print(f"Painel real: {len(painel):,}")
print(f"Balanceamento: {len(painel) / (n_ocup * n_periodos):.1%}")

# 2. Ocupações com poucos meses
ocup_meses = painel.groupby('cbo_4d')['periodo'].nunique()
print(f"\nMeses por ocupação:")
print(f"  Min: {ocup_meses.min()}, Max: {ocup_meses.max()}, Média: {ocup_meses.mean():.1f}")
print(f"  Ocupações com < 12 meses: {(ocup_meses < 12).sum()}")
print(f"  Ocupações com todos os {n_periodos} meses: {(ocup_meses == n_periodos).sum()}")

# 3. Estatísticas descritivas
print("\nEstatísticas descritivas:")
print(painel[['admissoes', 'desligamentos', 'saldo', 'salario_medio_adm']].describe().round(1))

# 4. Série temporal agregada
ts = painel.groupby('periodo_num').agg(
    total_adm=('admissoes', 'sum'),
    total_desl=('desligamentos', 'sum'),
    sal_medio=('salario_medio_adm', 'mean'),
).reset_index()
print("\nSérie temporal (primeiros e últimos 3 meses):")
print(ts.head(3).to_string(index=False))
print("...")
print(ts.tail(3).to_string(index=False))

CHECKPOINT — Painel Ocupação × Mês

Ocupações: 629
Períodos: 54
Painel teórico (balanceado): 33,966
Painel real: 32,988
Balanceamento: 97.1%

Meses por ocupação:
  Min: 2, Max: 54, Média: 52.4
  Ocupações com < 12 meses: 9
  Ocupações com todos os 54 meses: 589

Estatísticas descritivas:
       admissoes  desligamentos    saldo  salario_medio_adm
count    32988.0        32988.0  32988.0            32988.0
mean      3153.5         2873.6    279.9             6131.1
std      13880.0        12450.5   2323.0           158475.4
min          0.0            0.0 -46650.0                0.0
25%         60.0           60.0    -16.0             1782.8
50%        293.0          289.0      7.0             2371.3
75%       1338.0         1264.0    107.0             3811.2
max     289900.0       284338.0  90556.0         18799538.2

Série temporal (primeiros e últimos 3 meses):
 periodo_num  total_adm  total_desl   sal_medio
      202101    1550075     1293016 3697.157086
      202102    1715425     

### 4a. Crosswalk CBO 2002 → ISCO-08

Mapear os códigos CBO 2002 (usados no CAGED) para ISCO-08 (usados no índice ILO). **Esta é a etapa metodologicamente mais delicada do pipeline.**

#### Contexto

A CBO 2002 foi construída com base na ISCO-88/ISCO-08, compartilhando a mesma estrutura hierárquica:

| Nível | CBO 2002 | ISCO-08 | Alinhamento |
|-------|----------|---------|-------------|
| 1 dígito | Grande Grupo (10) | Major Group (10) | Perfeito |
| 2 dígitos | Subgrupo Principal (~46) | Sub-major Group (43) | Bom (14 CBOs sem match direto) |
| 3 dígitos | Subgrupo (~194) | Minor Group (130) | Parcial (~45% direto) |
| 4 dígitos | Família (~629) | Unit Group (427) | Divergente (~28% direto) |

#### Estratégia adotada: Dual (2d principal + 4d robustez)

**PARTE A — Especificação PRINCIPAL (2 dígitos):**
- CBO 2d → ISCO-08 Sub-major Group (match direto)
- Fallback: CBO 1d → ISCO-08 Major Group (média)
- Cobertura esperada: **100%**

**PARTE B — Especificação de ROBUSTEZ (4 dígitos, fallback hierárquico em 6 níveis):**

| Nível | Estratégia | Cobertura esperada |
|-------|------------|--------------------|
| N1 | CBO 4d = ISCO-08 4d (match direto) | ~28% |
| N2 | CBO 4d = ISCO-88 4d → ISCO-08 via correspondência oficial | ~+9% |
| N3 | CBO 3d = ISCO-08 3d (média Minor Group) | ~+20% |
| N4 | CBO 3d = ISCO-88 3d → ISCO-08 via correspondência | ~+1% |
| N5 | CBO 2d = ISCO-08 2d (= especificação principal) | ~+18% |
| N6 | CBO 1d = ISCO-08 1d (média Major Group) | ~+24% |

> **Nota — Muendler (CBO 1994):** O arquivo `cbo-isco-conc.csv` de Muendler & Poole (2004) mapeia CBO **1994** → ISCO-88, NÃO a CBO 2002 usada no CAGED. Por isso, o match 4d via Muendler é limitado. A estratégia principal utiliza a similaridade estrutural entre CBO 2002 e ISCO-08/88 com fallback hierárquico.

> **Nota sobre atenuação:** Se o crosswalk a 4 dígitos introduz erro de medição, o efeito típico é **atenuação** (viés em direção a zero). Encontrar efeito significativo mesmo com erro de medição sugere que o efeito real é provavelmente maior.

In [6]:
# Etapa 2a.4a — Crosswalk CBO 2002 → ISCO-08 (Dual: 2d principal + 4d robustez)
# Checkpoint: se o painel com crosswalk já existe, carrega direto.

if PAINEL_CROSSWALK_FILE.exists():
    painel = pd.read_parquet(PAINEL_CROSSWALK_FILE)
    print(f"Crosswalk carregado do checkpoint: {PAINEL_CROSSWALK_FILE.name}")
    print(f"  {len(painel):,} linhas, cobertura 2d: {painel['exposure_score_2d'].notna().mean():.1%}, "
          f"4d: {painel['exposure_score_4d'].notna().mean():.1%}")
else:
    # ══════════════════════════════════════════════════════════════════════
    # Carregar dados de referência
    # ══════════════════════════════════════════════════════════════════════
    df_ilo = pd.read_csv(ILO_FILE)
    df_ilo['isco_08_str'] = df_ilo['isco_08'].astype(str).str.zfill(4)
    print(f"Índice ILO carregado: {len(df_ilo)} ocupações ISCO-08")
    print(f"  Score range: [{df_ilo['exposure_score'].min():.3f}, {df_ilo['exposure_score'].max():.3f}]")

    # Dicts ILO em múltiplos níveis
    codes = df_ilo['isco_08_str']
    ilo_4d = df_ilo.groupby('isco_08_str')['exposure_score'].mean().to_dict()
    ilo_3d = df_ilo.assign(g=codes.str[:3]).groupby('g')['exposure_score'].mean().to_dict()
    ilo_2d = df_ilo.assign(g=codes.str[:2]).groupby('g')['exposure_score'].mean().to_dict()
    ilo_1d = df_ilo.assign(g=codes.str[:1]).groupby('g')['exposure_score'].mean().to_dict()

    # Correspondência ISCO-08 ↔ ISCO-88 (arquivo local)
    isco88_to_08 = {}
    isco88_3d_to_08_3d = {}
    if ISCO_08_88_FILE.exists():
        df_corr = pd.read_excel(ISCO_08_88_FILE, sheet_name='ISCO-08 to 88')
        df_corr['isco08_4d'] = df_corr['ISCO-08 code'].astype(str).str.strip().str.zfill(4)
        df_corr['isco88_4d'] = df_corr['ISCO-88 code'].astype(str).str.strip().str.zfill(4)
        isco88_to_08 = df_corr.groupby('isco88_4d')['isco08_4d'].apply(list).to_dict()
        df_corr['isco88_3d'] = df_corr['isco88_4d'].str[:3]
        df_corr['isco08_3d'] = df_corr['isco08_4d'].str[:3]
        isco88_3d_to_08_3d = df_corr.groupby('isco88_3d')['isco08_3d'].apply(
            lambda x: list(set(x))).to_dict()
        print(f"  Correspondência ISCO-08↔88: {len(df_corr)} mapeamentos")

    # ══════════════════════════════════════════════════════════════════════
    # PARTE A: 2 dígitos (PRINCIPAL)
    # ══════════════════════════════════════════════════════════════════════
    print(f"\n{'='*60}\nPARTE A: Crosswalk 2 dígitos (PRINCIPAL)\n{'='*60}")

    painel['exposure_score_2d'] = painel['cbo_2d'].map(ilo_2d)
    painel['match_level_2d'] = np.where(painel['exposure_score_2d'].notna(), '2-digit', None)

    # Fallback a 1 dígito
    mask_na = painel['exposure_score_2d'].isna()
    if mask_na.any():
        cbo_1d = painel.loc[mask_na, 'cbo_4d'].str[:1]
        painel.loc[mask_na, 'exposure_score_2d'] = cbo_1d.map(ilo_1d).values
        painel.loc[mask_na, 'match_level_2d'] = '1-digit (fallback)'

    painel['exposure_score'] = painel['exposure_score_2d']
    cov_2d = painel['exposure_score_2d'].notna().mean()
    print(f"  COBERTURA 2d: {cov_2d:.1%}")
    for lvl, cnt in painel['match_level_2d'].value_counts().items():
        print(f"    {lvl}: {cnt:,} ({cnt/len(painel):.1%})")

    # ══════════════════════════════════════════════════════════════════════
    # PARTE B: 4 dígitos (ROBUSTEZ) — fallback hierárquico 6 níveis
    # ══════════════════════════════════════════════════════════════════════
    print(f"\n{'='*60}\nPARTE B: Crosswalk 4 dígitos (ROBUSTEZ)\n{'='*60}")

    cbos_unicos = sorted(painel['cbo_4d'].unique())
    cbo_score_4d = {}
    cbo_match_level = {}
    counts = {'N1': 0, 'N2': 0, 'N3': 0, 'N4': 0, 'N5': 0, 'N6': 0, 'sem': 0}

    for cbo in cbos_unicos:
        score, level = None, None

        # N1: CBO 4d = ISCO-08 4d
        if cbo in ilo_4d:
            score, level = ilo_4d[cbo], 'N1: ISCO-08 4d direto'
            counts['N1'] += 1
        # N2: CBO 4d = ISCO-88 4d → ISCO-08
        if score is None and cbo in isco88_to_08:
            scores_c = [ilo_4d[c] for c in isco88_to_08[cbo] if c in ilo_4d]
            if scores_c:
                score, level = np.mean(scores_c), 'N2: via ISCO-88→08 4d'
                counts['N2'] += 1
        # N3: CBO 3d = ISCO-08 3d
        if score is None and cbo[:3] in ilo_3d:
            score, level = ilo_3d[cbo[:3]], 'N3: ISCO-08 3d'
            counts['N3'] += 1
        # N4: CBO 3d = ISCO-88 3d → ISCO-08 3d
        if score is None and cbo[:3] in isco88_3d_to_08_3d:
            scores_c = [ilo_3d[c] for c in isco88_3d_to_08_3d[cbo[:3]] if c in ilo_3d]
            if scores_c:
                score, level = np.mean(scores_c), 'N4: via ISCO-88→08 3d'
                counts['N4'] += 1
        # N5: CBO 2d = ISCO-08 2d
        if score is None and cbo[:2] in ilo_2d:
            score, level = ilo_2d[cbo[:2]], 'N5: ISCO-08 2d'
            counts['N5'] += 1
        # N6: CBO 1d = ISCO-08 1d
        if score is None and cbo[:1] in ilo_1d:
            score, level = ilo_1d[cbo[:1]], 'N6: ISCO-08 1d'
            counts['N6'] += 1

        if score is not None:
            cbo_score_4d[cbo] = score
            cbo_match_level[cbo] = level
        else:
            counts['sem'] += 1

    painel['exposure_score_4d'] = painel['cbo_4d'].map(cbo_score_4d)
    painel['match_level_4d'] = painel['cbo_4d'].map(cbo_match_level)

    total = len(cbos_unicos)
    print(f"  CBOs 4d únicos: {total}")
    for k, v in counts.items():
        if v > 0:
            print(f"    {k}: {v} ({v/total:.1%})")
    print(f"  COBERTURA 4d: {painel['exposure_score_4d'].notna().mean():.1%}")

    # Correlação 2d vs 4d
    df_check = painel[['cbo_4d', 'exposure_score_2d', 'exposure_score_4d']].drop_duplicates('cbo_4d')
    corr = df_check['exposure_score_2d'].corr(df_check['exposure_score_4d'])
    print(f"\n  Correlação Pearson (2d vs 4d): {corr:.4f}")

    painel.to_parquet(PAINEL_CROSSWALK_FILE, index=False)
    print(f"\nSalvo: {PAINEL_CROSSWALK_FILE.name}")

Crosswalk carregado do checkpoint: painel_caged_crosswalk.parquet
  32,988 linhas, cobertura 2d: 100.0%, 4d: 100.0%


### 4b. Verificar crosswalk (CHECKPOINT)

Validar qualidade do crosswalk nas DUAS especificações: principal (2 dígitos) e robustez (4 dígitos). Verificar cobertura, distribuição de scores, correlação entre especificações e sanity check por grande grupo CBO.

**Critérios de aceite:**
- Cobertura 2d ≥ 95% (esperado ~100%)
- Cobertura 4d ≥ 80% (esperado ~100% com fallback)
- Correlação 2d vs 4d > 0.8 (consistência entre especificações)

In [7]:
# Etapa 2a.4b — CHECKPOINT: Verificar crosswalk CBO → ISCO-08

print("=" * 60)
print("CHECKPOINT — Crosswalk CBO → ISCO-08 (Dual)")
print("=" * 60)

# 1. Cobertura
coverage_2d = painel['exposure_score_2d'].notna().mean()
coverage_4d = painel['exposure_score_4d'].notna().mean()
print(f"\n--- Cobertura ---")
print(f"  2 dígitos (PRINCIPAL): {coverage_2d:.1%}")
print(f"  4 dígitos (ROBUSTEZ):  {coverage_4d:.1%}")
if coverage_2d < 0.95:
    print(f"  ALERTA: Cobertura 2d abaixo de 95%!")
if coverage_4d < 0.80:
    print(f"  AVISO: Cobertura 4d abaixo de 80%.")

# 2. Estatísticas dos scores
print(f"\n--- Estatísticas dos scores ---")
print(f"\nexposure_score_2d (PRINCIPAL):")
print(painel['exposure_score_2d'].describe().round(4))
print(f"\nexposure_score_4d (ROBUSTEZ):")
print(painel['exposure_score_4d'].describe().round(4))

# 3. Correlação
mask_both = painel['exposure_score_2d'].notna() & painel['exposure_score_4d'].notna()
if mask_both.any():
    corr = painel.loc[mask_both, 'exposure_score_2d'].corr(
        painel.loc[mask_both, 'exposure_score_4d'])
    print(f"\n--- Correlação 2d vs 4d ---")
    print(f"  Pearson: {corr:.4f}")
    print(f"  {'Alta correlação — bom sinal de consistência.' if corr > 0.8 else 'Correlação moderada.'}")

# 4. Sanity check por grande grupo CBO
painel['grande_grupo_cbo'] = painel['cbo_4d'].str[0]
print(f"\n--- Exposição por grande grupo CBO ---")
print(f"{'Grande Grupo':<40} {'Score 2d':>10} {'Score 4d':>10}")
print("-" * 62)
for gg, nome in sorted(GRANDES_GRUPOS_CBO.items()):
    mask = painel['grande_grupo_cbo'] == gg
    if mask.any():
        s2d = painel.loc[mask, 'exposure_score_2d'].mean()
        s4d = painel.loc[mask, 'exposure_score_4d'].mean()
        s4d_str = f"{s4d:.3f}" if not np.isnan(s4d) else "N/A"
        flag = " (!)" if not np.isnan(s4d) and abs(s2d - s4d) > 0.1 else ""
        print(f"  {nome:<38} {s2d:>10.3f} {s4d_str:>10}{flag}")
print(f"  (!) = diferença > 0.1 entre 2d e 4d")

CHECKPOINT — Crosswalk CBO → ISCO-08 (Dual)

--- Cobertura ---
  2 dígitos (PRINCIPAL): 100.0%
  4 dígitos (ROBUSTEZ):  100.0%

--- Estatísticas dos scores ---

exposure_score_2d (PRINCIPAL):
count    32988.0000
mean         0.2778
std          0.1243
min          0.1167
25%          0.1658
50%          0.2459
75%          0.3725
max          0.6325
Name: exposure_score_2d, dtype: float64

exposure_score_4d (ROBUSTEZ):
count    32988.0000
mean         0.2830
std          0.1315
min          0.0900
25%          0.1658
50%          0.2500
75%          0.3650
max          0.7000
Name: exposure_score_4d, dtype: float64

--- Correlação 2d vs 4d ---
  Pearson: 0.9150
  Alta correlação — bom sinal de consistência.

--- Exposição por grande grupo CBO ---
Grande Grupo                               Score 2d   Score 4d
--------------------------------------------------------------
  Dirigentes                                  0.367      0.375
  Profissionais das ciências                  0.393   

### 5a. Definição de tratamento

Definir as variáveis de tratamento para a análise DiD. O tratamento é baseado na **exposição ocupacional à IA generativa**: ocupações com alta exposição (top 20%) vs. baixa exposição.

#### Variáveis criadas

| Variável | Definição | Uso |
|----------|-----------|-----|
| `alta_exp` | 1 se `exposure_score_2d >= percentil 80` | **Especificação principal** |
| `alta_exp_10` | 1 se `exposure_score_2d >= percentil 90` | Robustez (cutoff) |
| `alta_exp_25` | 1 se `exposure_score_2d >= percentil 75` | Robustez (cutoff) |
| `alta_exp_mediana` | 1 se `exposure_score_2d >= mediana` | Alternativa binária |
| `quintil_exp` | Quintil de exposição (Q1–Q5) | Análise por quantil |
| `alta_exp_4d` | 1 se `exposure_score_4d >= percentil 80` | Robustez (crosswalk 4d) |
| `did` | `post × alta_exp` | Interação DiD principal |
| `did_4d` | `post × alta_exp_4d` | Interação DiD robustez |

> **Nota:** Os thresholds são calculados sobre a distribuição de **ocupações** (uma obs por CBO), não ponderada por volume de movimentações. Cada ocupação tem peso igual na definição do tratamento.

> **Nota — Tratamento contínuo:** Além das dummies, `exposure_score_2d` e `exposure_score_4d` podem ser usados diretamente como tratamento contínuo em especificações alternativas, conforme Hui et al. (2024).

In [8]:
# Etapa 2a.5a — Definição de tratamento

# ── Thresholds sobre a distribuição de ocupações (2d) ──
ocup_scores_2d = painel.groupby('cbo_4d')['exposure_score_2d'].first().dropna()

thresholds_2d = {
    'alta_exp_10':      ocup_scores_2d.quantile(0.90),
    'alta_exp':         ocup_scores_2d.quantile(0.80),  # PRINCIPAL
    'alta_exp_25':      ocup_scores_2d.quantile(0.75),
    'alta_exp_mediana':  ocup_scores_2d.quantile(0.50),
}

print("Thresholds de exposição (2d, PRINCIPAL):")
for name, val in thresholds_2d.items():
    n_above = (ocup_scores_2d >= val).sum()
    pct = n_above / len(ocup_scores_2d) * 100
    print(f"  {name}: {val:.4f} ({n_above} ocupações, {pct:.0f}%)")

# ── Dummies de tratamento 2d ──
for name, threshold in thresholds_2d.items():
    painel[name] = (painel['exposure_score_2d'] >= threshold).astype(int)

# Quintis
painel['quintil_exp'] = pd.qcut(
    painel['exposure_score_2d'].rank(method='first'),
    q=5,
    labels=['Q1 (Baixa)', 'Q2', 'Q3', 'Q4', 'Q5 (Alta)']
)

# ── Dummies 4d (ROBUSTEZ) ──
ocup_scores_4d = painel.groupby('cbo_4d')['exposure_score_4d'].first().dropna()
threshold_4d_80 = ocup_scores_4d.quantile(0.80)
painel['alta_exp_4d'] = (painel['exposure_score_4d'] >= threshold_4d_80).astype(int)
print(f"\nThreshold 4d (p80): {threshold_4d_80:.4f} ({(ocup_scores_4d >= threshold_4d_80).sum()} ocupações)")

# ── Interações DiD ──
painel['did'] = painel['post'] * painel['alta_exp']
painel['did_4d'] = painel['post'] * painel['alta_exp_4d']

# ── Resumo ──
print(f"\n--- Distribuição de tratamento ---")
print(f"  Alta exp 2d (top 20%): {painel['alta_exp'].mean():.1%} das obs")
print(f"  Alta exp 4d (top 20%): {painel['alta_exp_4d'].mean():.1%} das obs")
print(f"  Períodos pré:  {painel[painel['post']==0].shape[0]:,}")
print(f"  Períodos pós:  {painel[painel['post']==1].shape[0]:,}")

concordancia = (painel['alta_exp'] == painel['alta_exp_4d']).mean()
print(f"  Concordância 2d vs 4d: {concordancia:.1%}")

# Tabela de contingência
ct = pd.crosstab(
    painel['post'].map({0: 'Pré', 1: 'Pós'}),
    painel['alta_exp'].map({0: 'Controle', 1: 'Tratamento'}),
    margins=True
)
print(f"\nTabela de contingência (2d, principal):")
print(ct)

Thresholds de exposição (2d, PRINCIPAL):
  alta_exp_10: 0.4433 (78 ocupações, 12%)
  alta_exp: 0.3854 (131 ocupações, 21%)
  alta_exp_25: 0.3725 (165 ocupações, 26%)
  alta_exp_mediana: 0.2459 (332 ocupações, 53%)

Threshold 4d (p80): 0.3863 (137 ocupações)

--- Distribuição de tratamento ---
  Alta exp 2d (top 20%): 20.3% das obs
  Alta exp 4d (top 20%): 21.3% das obs
  Períodos pré:  14,058
  Períodos pós:  18,930
  Concordância 2d vs 4d: 93.1%

Tabela de contingência (2d, principal):
alta_exp  Controle  Tratamento    All
post                                 
Pré          11195        2863  14058
Pós          15092        3838  18930
All          26287        6701  32988


### 5b. Verificar tratamento (CHECKPOINT)

Validar a definição de tratamento: top/bottom ocupações por exposição, distribuição por quintil, e concordância entre especificações 2d e 4d.


In [9]:
# Etapa 2a.5b — CHECKPOINT: Verificar definição de tratamento

print("=" * 60)
print("CHECKPOINT — Definição de Tratamento")
print("=" * 60)

# 1. Top 10 ocupações mais expostas
print("\n--- Top 10 ocupações MAIS expostas ---")
top10 = painel.groupby('cbo_4d').agg(
    exposure=('exposure_score', 'first'),
    admissoes_total=('admissoes', 'sum'),
).nlargest(10, 'exposure')
for cbo, row in top10.iterrows():
    nome = GRANDES_GRUPOS_CBO.get(cbo[0], '')
    print(f"  CBO {cbo}: score={row['exposure']:.3f}, admissões={row['admissoes_total']:,.0f}  ({nome})")

# 2. Bottom 10 ocupações menos expostas
print(f"\n--- 10 ocupações MENOS expostas ---")
bot10 = painel.groupby('cbo_4d').agg(
    exposure=('exposure_score', 'first'),
    admissoes_total=('admissoes', 'sum'),
).nsmallest(10, 'exposure')
for cbo, row in bot10.iterrows():
    nome = GRANDES_GRUPOS_CBO.get(cbo[0], '')
    print(f"  CBO {cbo}: score={row['exposure']:.3f}, admissões={row['admissoes_total']:,.0f}  ({nome})")

# 3. Distribuição por quintil
print(f"\n--- Estatísticas por quintil de exposição ---")
for q in ['Q1 (Baixa)', 'Q2', 'Q3', 'Q4', 'Q5 (Alta)']:
    sub = painel[painel['quintil_exp'] == q]
    if len(sub) > 0:
        print(f"  {q}: n={len(sub):,}, "
              f"exposure={sub['exposure_score'].mean():.3f}, "
              f"adm_mean={sub['admissoes'].mean():.0f}, "
              f"sal_medio={sub['salario_medio_adm'].mean():,.0f}")

# 4. Concordância
concordancia = (painel['alta_exp'] == painel['alta_exp_4d']).mean()
print(f"\n--- Concordância 2d vs 4d: {concordancia:.1%} ---")

print(f"\n{'=' * 60}")
print(f"CHECKPOINT CONCLUÍDO")
print(f"{'=' * 60}")

CHECKPOINT — Definição de Tratamento

--- Top 10 ocupações MAIS expostas ---
  CBO 4101: score=0.632, admissões=343,660  (Trabalhadores de serv. admin.)
  CBO 4102: score=0.632, admissões=109,397  (Trabalhadores de serv. admin.)
  CBO 4110: score=0.632, admissões=7,169,112  (Trabalhadores de serv. admin.)
  CBO 4121: score=0.632, admissões=36,099  (Trabalhadores de serv. admin.)
  CBO 4122: score=0.632, admissões=217,727  (Trabalhadores de serv. admin.)
  CBO 4131: score=0.632, admissões=521,115  (Trabalhadores de serv. admin.)
  CBO 4132: score=0.632, admissões=178,605  (Trabalhadores de serv. admin.)
  CBO 4141: score=0.632, admissões=4,019,302  (Trabalhadores de serv. admin.)
  CBO 4142: score=0.632, admissões=406,552  (Trabalhadores de serv. admin.)
  CBO 4151: score=0.632, admissões=36,816  (Trabalhadores de serv. admin.)

--- 10 ocupações MENOS expostas ---
  CBO 9101: score=0.117, admissões=44,866  (Manutenção e reparação)
  CBO 9102: score=0.117, admissões=13,108  (Manutenção e

### 6a. Enriquecimento do painel (variáveis adicionais)

Adicionar variáveis de controle e contexto temporal ao painel para a análise DiD.

| Variável | Cálculo | Uso |
|----------|---------|-----|
| `tempo_relativo_meses` | Meses desde Dez/2022 (t=0) | Event study |
| `trend` | Tendência linear (0, 1, 2, ...) | Controle de tendência |
| `mes_do_ano` | Mês do ano (1-12) | Dummies de sazonalidade |
| `salario_sm` | `salario_medio_adm / SM do ano` | Normalização salarial |
| `grande_grupo_nome` | Nome do grande grupo CBO | Efeitos fixos |

In [10]:
# Etapa 2a.6a — Enriquecimento do painel

def periodo_num_to_months(pn):
    """Converter periodo_num (YYYYMM) para contagem absoluta de meses."""
    return (pn // 100) * 12 + (pn % 100)

# ── Tempo relativo ao tratamento ──
ref_periodo = ANO_TRATAMENTO * 100 + MES_TRATAMENTO
painel['meses_abs'] = painel['periodo_num'].apply(periodo_num_to_months)
ref_meses = periodo_num_to_months(ref_periodo)
painel['tempo_relativo_meses'] = painel['meses_abs'] - ref_meses

print(f"Tempo relativo: [{painel['tempo_relativo_meses'].min()}, "
      f"{painel['tempo_relativo_meses'].max()}] meses")
print(f"Referência (t=0): {MES_TRATAMENTO}/{ANO_TRATAMENTO}")

# ── Tendência temporal e sazonalidade ──
painel['trend'] = painel['meses_abs'] - painel['meses_abs'].min()
painel['mes_do_ano'] = painel['mes'].astype(int)

# ── Normalização salarial ──
painel['sm_ano'] = painel['ano'].astype(int).map(SALARIO_MINIMO)
painel['salario_sm'] = painel['salario_medio_adm'] / painel['sm_ano']
painel['ln_salario_sm'] = np.log(painel['salario_sm'].clip(lower=0.1))

# ── Grande grupo ocupacional ──
painel['grande_grupo_cbo'] = painel['cbo_4d'].str[0]
painel['grande_grupo_nome'] = painel['grande_grupo_cbo'].map(GRANDES_GRUPOS_CBO)

print(f"\nVariáveis adicionadas: tempo_relativo_meses, trend, mes_do_ano, "
      f"salario_sm, ln_salario_sm, grande_grupo_nome")
print(f"Colunas totais: {painel.shape[1]}")

Tempo relativo: [-23, 30] meses
Referência (t=0): 12/2022

Variáveis adicionadas: tempo_relativo_meses, trend, mes_do_ano, salario_sm, ln_salario_sm, grande_grupo_nome
Colunas totais: 42


### 7. Salvar dataset analítico final

Selecionar colunas finais, remover ocupações sem score de exposição e salvar o dataset pronto para a análise DiD (Notebook 2b).

**Saída:**
- `data/output/painel_caged_did_ready.parquet` — formato eficiente para análise
- `data/output/painel_caged_did_ready.csv` — backup legível

In [11]:
# Etapa 2a.7 — Salvar dataset analítico final

# ── Selecionar colunas finais ──
cols_finais = [
    # Identificação
    'cbo_4d', 'cbo_2d', 'ano', 'mes', 'periodo', 'periodo_num',
    # Outcomes
    'admissoes', 'desligamentos', 'saldo', 'n_movimentacoes',
    'ln_admissoes', 'ln_desligamentos',
    'salario_medio_adm', 'salario_mediano_adm', 'salario_medio_desl',
    'ln_salario_adm', 'salario_sm', 'ln_salario_sm',
    # Demografia das admissões
    'idade_media_adm', 'pct_mulher_adm', 'pct_superior_adm',
    # Exposição IA — DUAL
    'exposure_score_2d',   # PRINCIPAL
    'exposure_score_4d',   # ROBUSTEZ
    # Tratamento — DUAL
    'alta_exp',            # Top 20% score 2d (PRINCIPAL)
    'alta_exp_10', 'alta_exp_25', 'alta_exp_mediana', 'quintil_exp',
    'alta_exp_4d',         # Top 20% score 4d (ROBUSTEZ)
    # Temporal
    'post', 'did', 'did_4d', 'tempo_relativo_meses', 'trend', 'mes_do_ano',
    # Classificação
    'grande_grupo_cbo', 'grande_grupo_nome',
]

cols_existentes = [c for c in cols_finais if c in painel.columns]
cols_faltantes = [c for c in cols_finais if c not in painel.columns]
if cols_faltantes:
    print(f"AVISO: Colunas não encontradas: {cols_faltantes}")

painel_final = painel[cols_existentes].copy()

# ── Remover ocupações sem score principal (2d) ──
n_antes = len(painel_final)
painel_final = painel_final[painel_final['exposure_score_2d'].notna()]
n_depois = len(painel_final)
if n_antes > n_depois:
    print(f"Removidas {n_antes - n_depois:,} linhas sem exposure_score_2d")

# ── Salvar ──
painel_final.to_parquet(PAINEL_FINAL_PARQUET, index=False)
painel_final.to_csv(PAINEL_FINAL_CSV, index=False)

# ══════════════════════════════════════════════════════════════════════
# RESUMO FINAL
# ══════════════════════════════════════════════════════════════════════
print(f"\n{'=' * 60}")
print("DATASET ANALÍTICO FINAL — ETAPA 2a")
print(f"{'=' * 60}")
print(f"  Observações:        {len(painel_final):,}")
print(f"  Ocupações (CBO 4d): {painel_final['cbo_4d'].nunique()}")
print(f"  Períodos:           {painel_final['periodo'].nunique()} meses")
print(f"    Pré-tratamento:   {painel_final[painel_final['post']==0]['periodo'].nunique()}")
print(f"    Pós-tratamento:   {painel_final[painel_final['post']==1]['periodo'].nunique()}")
print(f"  Cobertura 2d:       {painel_final['exposure_score_2d'].notna().mean():.1%}")
print(f"  Cobertura 4d:       {painel_final['exposure_score_4d'].notna().mean():.1%}")
print(f"  Tratamento 2d:      {painel_final['alta_exp'].mean():.1%} das obs")
print(f"  Tratamento 4d:      {painel_final['alta_exp_4d'].mean():.1%} das obs")
print(f"  Colunas:            {painel_final.shape[1]}")
print(f"\n  Salvo em:")
print(f"    {PAINEL_FINAL_PARQUET}")
print(f"    {PAINEL_FINAL_CSV}")
pq_mb = PAINEL_FINAL_PARQUET.stat().st_size / 1e6
csv_mb = PAINEL_FINAL_CSV.stat().st_size / 1e6
print(f"    Tamanho: {pq_mb:.1f} MB (parquet), {csv_mb:.1f} MB (csv)")

print(f"\n  Info:")
painel_final.info()


DATASET ANALÍTICO FINAL — ETAPA 2a
  Observações:        32,988
  Ocupações (CBO 4d): 629
  Períodos:           54 meses
    Pré-tratamento:   23
    Pós-tratamento:   31
  Cobertura 2d:       100.0%
  Cobertura 4d:       100.0%
  Tratamento 2d:      20.3% das obs
  Tratamento 4d:      21.3% das obs
  Colunas:            37

  Salvo em:
    data/output/painel_caged_did_ready.parquet
    data/output/painel_caged_did_ready.csv
    Tamanho: 2.8 MB (parquet), 9.9 MB (csv)

  Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32988 entries, 0 to 32987
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   cbo_4d                32988 non-null  object  
 1   cbo_2d                32988 non-null  object  
 2   ano                   32988 non-null  Int64   
 3   mes                   32988 non-null  Int64   
 4   periodo               32988 non-null  object  
 5   periodo_num           32988 non-null  i

### Limitações desta etapa

1. **Novo CAGED (descontinuidade 2020):** A transição para o eSocial (2020) pode afetar a comparabilidade. Mitigamos ao iniciar em 2021 (eSocial já estabilizado, sem efeitos COVID).

2. **Fluxos vs. estoques:** O CAGED mede movimentações (admissões/desligamentos), não o estoque de empregados. Quedas em admissões não significam necessariamente queda no emprego total — podem refletir menor rotatividade. Esta é a mesma lógica usada por Hui et al. (2024) com dados do Upwork.

3. **Crosswalk CBO → ISCO-08 (especificação principal, 2 dígitos):** Ao agregar por Sub-major Group com fallback a Major Group, perdemos variação intragrupo. Ocupações diferentes dentro do mesmo grupo recebem o mesmo score. A especificação de robustez a 4 dígitos (com fallback hierárquico em 6 níveis) ajuda a avaliar se essa agregação afeta os resultados.

4. **Crosswalk CBO → ISCO-08 (robustez, 4 dígitos):** O match direto CBO 4d = ISCO-08 4d cobre apenas ~28% das ocupações. Para o restante, usamos fallback hierárquico (via correspondência ISCO-88→ISCO-08, médias a 3d, 2d e 1d). Quanto mais granular o match, mais preciso o score — mas mesmo com fallback, a correlação entre as especificações 2d e 4d é >0.91, indicando consistência. Erro de medição no tratamento tipicamente atenua os coeficientes (viés em direção a zero).

5. **Muendler: CBO 1994, não CBO 2002:** O arquivo de concordância Muendler & Poole (2004) mapeia a CBO *1994* (formato X-XX.XX), não a CBO 2002 (XXXX) usada no CAGED. A utilidade do Muendler para match 4d direto é limitada. A estratégia adotada usa a similaridade estrutural entre CBO 2002 e ISCO-08/88 (ambas baseadas na ISCO), combinada com a tabela oficial de correspondência ISCO-08↔ISCO-88.

6. **Emprego formal apenas:** O CAGED cobre apenas o mercado formal (CLT). A informalidade (~40% da força de trabalho brasileira) não é capturada. Efeitos da IA sobre o setor informal requerem fontes alternativas (PNAD).

7. **Índice global aplicado ao Brasil:** Mesma limitação da Etapa 1 — o índice ILO foi desenvolvido com foco global e pode não capturar especificidades do mercado de trabalho brasileiro.

---

### Checklist de entregáveis

- [x] `data/raw/caged_{ano}.parquet` — Microdados CAGED por ano (2021–2025)
- [x] `data/input/cbo-isco-conc.csv` — Concordância Muendler CBO 1994→ISCO-88
- [x] `data/input/Correspondência ISCO 08 a 88.xlsx` — Tabela oficial ISCO-08↔ISCO-88
- [x] `data/processed/ilo_exposure_clean.csv` — Índice ILO processado (reusado da Etapa 1)
- [x] `data/output/painel_caged_did_ready.parquet` — Dataset analítico final (com scores 2d e 4d)
- [x] `data/output/painel_caged_did_ready.csv` — Backup CSV
- [x] Todos os CHECKPOINTs passando sem warnings críticos
- [x] Cobertura crosswalk 2d = 100%
- [x] Cobertura crosswalk 4d = 100% (com fallback hierárquico)
- [x] Correlação entre scores 2d e 4d: 0.9147
- [x] Sanity check por grande grupo coerente com a literatura
